#   What is the best way to add thermodynamic information to an SBML model?

Moritz E. Beber, Elad Noor

COMBINE2020

## Motivation

* Metabolic Engineering
* Genome-scale constraint-based metabolic modeling
* Highly underdetermined systems
* Authors constrain models to fit their scenario
* Altering metabolic network or the medium can have unintended consequences

* Elad has published a lot both on theory and application

1. Niebel, Bastian, Simeon Leupold, and Matthias Heinemann. 2019. “An Upper Limit on Gibbs Energy Dissipation Governs Cellular Metabolism.” Nature Metabolism 1 (1): 125–32. https://doi.org/10.1038/s42255-018-0006-7.
2. Hädicke, Oliver, Axel von Kamp, Timur Aydogan, and Steffen Klamt. 2018. “OptMDFpathway: Identification of Metabolic Pathways with Maximal Thermodynamic Driving Force and Its Application for Analyzing the Endogenous CO2 Fixation Potential of Escherichia Coli.” PLOS Computational Biology 14 (9): e1006492. https://doi.org/10.1371/journal.pcbi.1006492.

## Thermodynamic Information

* [eQuilibrator](http://equilibrator.weizmann.ac.il/) provides:
    * standard formation energy $ \Delta_f G'^\circ $ and at physiological conditions $ \Delta_f G' $
    * standard Gibbs free energy $ \Delta_r G'^\circ $ and at physiological conditions $ \Delta_r G' $

### eQuilibrator Content

<img src="../images/metanetx-4.0.png" style="text-align: center; vertical-align: middle;" />

Python package [equilibrator-api](https://gitlab.com/equilibrator/equilibrator-api)

In [10]:
from equilibrator_api import ComponentContribution, Q_

In [11]:
cc = ComponentContribution()

Fragments already downloaded
Fragments already downloaded


In [12]:
reaction = cc.parse_reaction_formula("kegg:C00002 + kegg:C00001 = kegg:C00008 + kegg:C00009")
cc.p_h = Q_(7.4)
cc.p_mg = Q_(3.0)
cc.ionic_strength = Q_("0.25M")
cc.temperature = Q_("298.15K")

In [13]:
dG0_prime = cc.standard_dg_prime(reaction)
print(f"ΔG'° = {dG0_prime}")

ΔG'° = (-29.14 +/- 0.30) kilojoule / mole


In [14]:
dGm_prime = cc.physiological_dg_prime(reaction)
print(f"ΔG'm = {dGm_prime}")

ΔG'm = (-46.26 +/- 0.30) kilojoule / mole


In [15]:
concentrations = [('kegg:C00002', 1.0),
                  ('kegg:C00009', 0.1),
                  ('kegg:C00008', 3.0)]
for cid, conc in concentrations:
    compound = cc.get_compound(cid)
    abundance = Q_(conc, "mM")
    reaction.set_abundance(compound, abundance)

dG_prime = cc.dg_prime(reaction)
print(f"ΔG' = {dG_prime}")

ΔG' = (-49.24 +/- 0.30) kilojoule / mole


### Summary

* We can provide thermodynamic information for genome-scale models
* It is _crucial_ that we communicate covariance information to reduce uncertainties in networks of reactions

## How do we add this information to a model?

* Can easily provide scripts but for publishing and other communication would like to package this

* Conditions should be encoded on compartments
* Some energy values could be stored in the new SBML key value store
* Covariance is trickier:
    * either a matrix
    * or many individual vectors
* Does it make sense to encode constraints?    

* SBML array package?
* SBtab or ObjTable?
* COMBINE archive?